In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r'C:\Users\user\innio_case_study\data\northwind.db')
c_df = pd.read_sql_query("select * from customers", conn)
o_df = pd.read_sql_query("select * from orders", conn)
list_cities = c_df['City'].unique().tolist()
print(list_cities)

['Berlin', 'México D.F.', 'London', 'Luleå', 'Mannheim', 'Strasbourg', 'Madrid', 'Marseille', 'Tsawassen', 'Buenos Aires', 'Bern', 'Sao Paulo', 'Aachen', 'Nantes', 'Graz', 'Lille', 'Bräcke', 'München', 'Torino', 'Lisboa', 'Barcelona', 'Sevilla', 'Campinas', 'Eugene', 'Caracas', 'Rio de Janeiro', 'San Cristóbal', 'Elgin', 'Cork', 'Cowes', 'Brandenburg', 'Versailles', 'Toulouse', 'Vancouver', 'Walla Walla', 'Frankfurt a.M.', 'San Francisco', 'Barquisimeto', 'I. de Margarita', 'Portland', 'Bergamo', 'Bruxelles', 'Montréal', 'Leipzig', 'Anchorage', 'Köln', 'Paris', 'Salzburg', 'Cunewalde', 'Albuquerque', 'Reggio Emilia', 'Genève', 'Stavern', 'Boise', 'Kobenhavn', 'Lander', 'Charleroi', 'Butte', 'Münster', 'Kirkland', 'Århus', None, 'Lyon', 'Reims', 'Stuttgart', 'Oulu', 'Resende', 'Seattle', 'Helsinki', 'Warszawa']


In [41]:
conn2 = sqlite3.connect(r'C:\Users\user\innio_case_study\data\datawarehouse.db')
c_df.to_sql("bronze_customers", conn2, if_exists="replace", index=False)
o_df.to_sql("bronze_orders", conn2, if_exists="replace", index=False)


16282

In [135]:
import requests
import pandas as pd

api_key = "8de9abed422e492a58cf716b0e24caf0"

weather_data = []

for city in list_cities:
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Manual field extraction with safety checks
        weather_info = {
            'input_city': city,
            'api_city_name': data.get('name'),
            'country': data.get('sys', {}).get('country'),
            'lat': data.get('coord', {}).get('lat'),
            'lon': data.get('coord', {}).get('lon'),
            'temperature_C': data.get('main', {}).get('temp'),
            'feels_like_C': data.get('main', {}).get('feels_like'),
            'humidity_%': data.get('main', {}).get('humidity'),
            'pressure_hPa': data.get('main', {}).get('pressure'),
            'wind_speed_mps': data.get('wind', {}).get('speed'),
            'cloud_coverage_%': data.get('clouds', {}).get('all'),
            'weather_main': data.get('weather', [{}])[0].get('main'),
            'weather_description': data.get('weather', [{}])[0].get('description'),
        }

        weather_data.append(weather_info)
    else:
        print(f"Failed for {city}: Status {response.status_code}")

# Convert to DataFrame
weather_df = pd.DataFrame(weather_data)



Failed for México D.F.: Status 404
Failed for Tsawassen: Status 404
Failed for Frankfurt a.M.: Status 404
Failed for I. de Margarita: Status 404


In [26]:
weather_df = weather_df.rename(columns={'api_city_name': 'City'})  # Rename 'name' to 'City' for consistency


In [ ]:
weather_df['City'] = weather_df['City'].astype(str)  # Ensure 'City' is a string type

In [31]:
import unicodedata

def normalize(city):
    if pd.isna(city):
        return None
    # Remove accents, convert to lowercase, strip whitespace
    return ''.join(
        c for c in unicodedata.normalize('NFD', city)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

c_df['City'] = c_df['City'].apply(normalize)
weather_df['City'] = weather_df['City'].apply(normalize)

In [108]:
geo_df = c_df.merge(
    weather_df.set_index(['City']),
    left_on=['City'],
    right_index=True,
    how='left'
)

In [109]:
# Rename all columns in geo_df to PascalCase
geo_df.columns = [col.strip().replace('_', ' ').title().replace(' ', '') for col in geo_df.columns]

In [110]:
geo_df.rename(columns={
    "Customerid": "Customer_ID",
    "Companyname": "company_name",
    "Contactname": "contact_name",
    "Contacttitle": "contact_title",
    "Postalcode": "postal_code",
    "InputCity": "input_city",
    "TemperatureC": "temperature_c",
    "FeelsLikeC": "feels_like_c",
    "Humidity%": "humidity",
    "PressureHpa": "pressure_hpa",
    "WindSpeedMps": "wind_speed_mps",
    "CloudCoverage%": "cloud_coverage",
    "WeatherMain": "weather_main",
    "WeatherDescription": "weather_description"
}, inplace=True)

In [111]:
geo_df.head()  # Display the first few rows of the merged DataFrame

,Customer_ID,company_name,contact_name,contact_title,Address,City,Region,postal_code,Country,Phone,...,Lat,Lon,temperature_c,feels_like_c,humidity,pressure_hpa,wind_speed_mps,cloud_coverage,weather_main,weather_description
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,berlin,Western Europe,12209,Germany,030-0074321,...,52.5244,13.4105,22.57,22.71,70.0,1009.0,5.14,40.0,Clouds,scattered clouds
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,mexico d.f.,Central America,05021,Mexico,(5) 555-4729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,mexico d.f.,Central America,05023,Mexico,(5) 555-3932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,london,British Isles,WA1 1DP,UK,(171) 555-7788,...,51.5085,-0.1257,22.44,22.12,53.0,1020.0,3.60,40.0,Clouds,scattered clouds
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,lulea,Northern Europe,S-958 22,Sweden,0921-12 34 65,...,65.5842,22.1547,24.04,24.30,69.0,1008.0,2.57,0.0,Clear,clear sky


In [112]:
geo_df.drop(columns=["Fax", "input_city"], inplace=True)

In [113]:
geo_df["City"] = geo_df["City"].str.title().str.strip()
geo_df["has_weather"] = geo_df["Lat"].notnull()


In [114]:
# Convert all column names to PascalCase
geo_df.columns = [col.strip().replace('_', ' ').replace('%', 'Percent').replace('id', 'ID').title().replace(' ', '') for col in geo_df.columns]

In [115]:
# Drop only one 'Country' column from geo_df, keeping the first occurrence
geo_df = geo_df.loc[:, ~geo_df.columns.duplicated()]

In [116]:
geo_df.to_sql("silver_cust_weather", conn2, if_exists="replace", index=False)

93

In [118]:
orders_df = pd.read_sql_query("""
    SELECT CustomerID, OrderID, EmployeeID, OrderDate
    FROM bronze_orders
""", conn2)

full_join_df = pd.merge(orders_df, geo_df, left_on="CustomerID", right_on="CustomerId", how="left")


In [123]:
full_join_df = full_join_df.drop(columns=["CustomerId"])


In [124]:
full_join_df.to_sql("silver_cust_weat_order", conn2, if_exists="replace", index=False)

16282

In [50]:
region_df = pd.read_excel(r"C:\Users\user\innio_case_study\data\region_mapping.xlsx")

In [51]:
region_df.to_sql("bronze_region_mapping", conn2, if_exists="replace", index=False)

21

In [131]:
region_df.columns = [col.strip().lower().replace(" ", "_") for col in region_df.columns]
region_df.rename(columns={"country": "country_mapping"}, inplace=True)

# Normalize country values
region_df["country_mapping"] = region_df["country_mapping"].str.strip().str.lower()
full_join_df["Country"] = full_join_df["Country"].str.strip().str.lower()

# Join on country
final_df = pd.merge(
    full_join_df,
    region_df,
    left_on="Country",
    right_on="country_mapping",
    how="left"
)

# Add final region field (can choose which to prefer)
#final_df["region"] = final_df["region_starting_2016"].combine_first(final_df["region_until_2017"])

In [132]:
final_df["Final_region"] = final_df["region_starting_2016"].combine_first(final_df["region_until_2017"])

In [133]:
final_df.head()  # Display the first few rows of the final DataFrame

,CustomerID,OrderID,EmployeeID,OrderDate,CompanyName,ContactName,ContactTitle,Address,City,Region,...,PressureHpa,WindSpeedMps,CloudCoverage,WeatherMain,WeatherDescription,HasWeather,country_mapping,region_starting_2016,region_until_2017,Final_region
0,VINET,10248,5,2016-07-04,Vins et alcools Chevalier,Paul Henriot,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,...,NaN,NaN,NaN,NaN,NaN,False,france,France,Europe,France
1,TOMSP,10249,6,2016-07-05,Toms Spezialitäten,Karin Josephs,Marketing Manager,Luisenstr. 48,Munster,Western Europe,...,1015.0,2.57,40.0,Clouds,scattered clouds,True,germany,Germany,Europe,Germany
2,HANAR,10250,4,2016-07-08,Hanari Carnes,Mario Pontes,Accounting Manager,"Rua do Paço, 67",Rio De Janeiro,South America,...,1013.0,12.35,0.0,Clear,clear sky,True,brazil,Brazil,USA,Brazil
3,VICTE,10251,3,2016-07-08,Victuailles en stock,Mary Saveley,Sales Agent,"2, rue du Commerce",Lyon,Western Europe,...,NaN,NaN,NaN,NaN,NaN,False,france,France,Europe,France
4,SUPRD,10252,4,2016-07-09,Suprêmes délices,Pascale Cartrain,Accounting Manager,"Boulevard Tirou, 255",Charleroi,Western Europe,...,1019.0,2.57,40.0,Clouds,scattered clouds,True,belgium,France,Europe,France


In [162]:
%run extract.py \
    "C:\\Users\\user\\innio_case_study\\data\\northwind.db" \
    "C:\\Users\\user\\innio_case_study\\data\\warehouse.db" \
    "C:\\Users\\user\\innio_case_study\\data\\region_mapping.xlsx"


2025-07-29 00:32:09 [INFO] Connecting to database at C:\\Users\\user\\innio_case_study\\data\\northwind.db
2025-07-29 00:32:09 [INFO] Connecting to database at C:\\Users\\user\\innio_case_study\\data\\warehouse.db
2025-07-29 00:32:09 [INFO] Extracting 'Customers' into 'raw_customers'
2025-07-29 00:32:09 [INFO]   Retrieved 93 rows from Customers
2025-07-29 00:32:09 [INFO]   Loaded 93 rows into raw_customers
2025-07-29 00:32:09 [INFO] Extracting 'Orders' into 'raw_orders'
2025-07-29 00:32:09 [INFO]   Retrieved 16282 rows from Orders
2025-07-29 00:32:09 [INFO]   Loaded 16282 rows into raw_orders
2025-07-29 00:32:09 [INFO] Reading region mapping from 'C:\\Users\\user\\innio_case_study\\data\\region_mapping.xlsx'
2025-07-29 00:32:09 [INFO]   Read 21 rows from region mapping
2025-07-29 00:32:09 [INFO]   Loaded 21 rows into raw_region_mapping
2025-07-29 00:32:09 [INFO] Extract process completed successfully.


In [173]:
# In a Jupyter cell:
%run transform.py \
    C:\\Users\\user\\innio_case_study\\data\\warehouse.db \
    C:\\Users\\user\\innio_case_study\\data\\region_mapping.xlsx


2025-07-29 00:57:09 [INFO] Connecting to database at C:\\Users\\user\\innio_case_study\\data\\warehouse.db
2025-07-29 00:57:09 [INFO] Transform: building stg_order_customer
2025-07-29 00:57:09 [INFO] Loaded 16282 rows into stg_order_customer
2025-07-29 00:57:09 [INFO] Transform: building stg_order_customer_region
2025-07-29 00:57:09 [WARNING] 335 orders without region mapping
2025-07-29 00:57:09 [WARNING] 335 orders without region mapping
2025-07-29 00:57:09 [INFO] Loaded 16282 rows into stg_order_customer_region
2025-07-29 00:57:09 [INFO] Transform process completed successfully.


In [182]:
%run load.py C:\\Users\\user\\innio_case_study\\data\\warehouse.db 

2025-07-29 01:08:28 [INFO] Connecting to database at C:\\Users\\user\\innio_case_study\\data\\warehouse.db
2025-07-29 01:08:28 [INFO] Loading dimension table 'dim_customer'
2025-07-29 01:08:33 [ERROR] Load process failed
Traceback (most recent call last):
  File "c:\Users\user\innio_case_study\venv\Lib\site-packages\pandas\io\sql.py", line 2664, in execute
    cur.execute(sql, *args)
sqlite3.OperationalError: database is locked

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\user\innio_case_study\scripts\load.py", line 150, in main
    build_dim_customer(conn)
  File "C:\Users\user\innio_case_study\scripts\load.py", line 29, in build_dim_customer
    df.to_sql('dim_customer', conn, if_exists='replace', index=False)
  File "c:\Users\user\innio_case_study\venv\Lib\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\

SystemExit: 1

In [174]:
%run weather_api.py C:\\Users\\user\\innio_case_study\\data\\warehouse.db 8de9abed422e492a58cf716b0e24caf0

2025-07-29 00:57:13 [INFO] Connecting to database at C:\\Users\\user\\innio_case_study\\data\\warehouse.db
2025-07-29 00:57:13 [INFO] Found 70 unique cities
C:\Users\user\innio_case_study\scripts\weather_api.py:48: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'weather_timestamp': datetime.utcfromtimestamp(d.get('dt', 0))
2025-07-29 00:57:13 [INFO] Fetched weather for Berlin: 18.16°C, few clouds
2025-07-29 00:57:13 [WARNING] Failed for 'México D.F.': 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=M%C3%A9xico+D.F.&appid=8de9abed422e492a58cf716b0e24caf0&units=metric
2025-07-29 00:57:13 [INFO] Fetched weather for London: 18.75°C, broken clouds
2025-07-29 00:57:13 [INFO] Fetched weather for Luleå: 20.04°C, overcast clouds
2025-07-29 00:57:13 [INFO] Fetched weather

In [180]:
pd.read_sql("SELECT * FROM view_order_analysis LIMIT 5", conn)


ProgrammingError: Cannot operate on a closed database.